In [1]:
import gym
import time
import os
import numpy as np
import matplotlib.pyplot as plt
import sys
sys.path.insert(0, "../Environment/")
import halite_env as Env

In [2]:
from importlib import reload
reload(Env)

<module 'halite_env' from '../Environment/halite_env.py'>

In [3]:
num_players = 1
map_size = 7 # 7 x 7 map
HEnv = Env.HaliteEnv(num_players, map_size)

In [4]:
def Single_ship_env_initialization(map_size, num_players = 1):
    HEnv = Env.HaliteEnv(num_players, map_size)
    return Env.SingleShipEnv(HEnv, 1, map_size)

def check_position(env):
    print(env.state[:,:,:])


In [5]:
env = Single_ship_env_initialization(map_size)
_ = env.reset()

# Baselines

In [6]:
from stable_baselines.common.policies import MlpPolicy
from stable_baselines.common.vec_env import DummyVecEnv, SubprocVecEnv
from stable_baselines.common import set_global_seeds
from stable_baselines import PPO2, ACKTR
from stable_baselines.bench import Monitor
import stable_baselines
#reload(stable_baselines)
#from stable_baselines.logger import configure
#configure("/home/mango/Documents/haliteRL/tensorboard_ppo2", ['stdout','log','csv','tensorboard'])
from hyperparams_opt import hyperparam_optimization


MULTI = False
COMPARISON = False
obs_type = "raw"
n_timesteps = 100000
map_size = 7

In [7]:
def make_env(map_size, rank, seed=1234, log_dir=None):
    """
    Utility function for multiprocessed env.

    :param env_id: (str) the environment ID
    :param rank: (int) index of the subprocess
    :param seed: (int) the inital seed for RNG
    """
    if log_dir is None and log_dir != '':
        log_dir = "/tmp/gym/{}/".format(int(time.time()))
    os.makedirs(log_dir, exist_ok=True)

    def _init():
        set_global_seeds(seed + rank)
        env = Single_ship_env_initialization(map_size)
        
        env.seed(seed + rank)
        env = Monitor(env, os.path.join(log_dir, str(rank)), allow_early_resets=True)

        return env

    return _init

def evaluate(model, num_steps=1000):
    """
    Evaluate a RL agent
    :param model: (BaseRLModel object) the RL Agent
    :param num_steps: (int) number of timesteps to evaluate it
    :return: (float) Mean reward
    """
    episode_rewards = [[0.0] for _ in range(env.num_envs)]
    obs = env.reset()
    for i in range(num_steps):
        # _states are only useful when using LSTM policies
        actions, _states = model.predict(obs)
        # here, action, rewards and dones are arrays
        # because we are using vectorized env
        obs, rewards, dones, info = env.step(actions)

        # Stats
        for j in range(env.num_envs):
            episode_rewards[j][-1] += rewards[j]
            if dones[j]:
                episode_rewards[j].append(0.0)

    mean_rewards = [0.0 for _ in range(env.num_envs)]
    n_episodes = 0
    for i in range(env.num_envs):
        mean_rewards[i] = np.mean(episode_rewards[i])
        n_episodes += len(episode_rewards[i])

    # Compute mean reward
    mean_reward = round(np.mean(mean_rewards), 1)
    print("Mean reward:", mean_reward, "Num episodes:", n_episodes)

    return mean_reward





if MULTI:
    num_cpu = 32  # Number of processes to use
    # Create the vectorized environment
    env = SubprocVecEnv([make_env(map_size, i) for i in range(num_cpu)])
else:
    # env = SingleSnek(obs_type="rgb", n_food=3)
    env = Single_ship_env_initialization(map_size)
    # The algorithms require a vectorized environment to run
    env = DummyVecEnv([lambda: env])


ob = env.reset()


In [8]:
import yaml

with open('hyperparams.yml', 'r') as f:
            hyperparams_dict = yaml.load(f)
            hyperparams = hyperparams_dict['halite']

print(hyperparams)
print(hyperparams.get('n_envs', 1))
print(hyperparams)

#hyperparams.get('n_envs')

/home/mango/.local/lib/python3.6/site-packages/ipykernel_launcher.py:4: YAMLLoadWarning:

calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.



{'policy': 'MlpPolicy', 'n_envs': 8, 'n_steps': 128, 'noptepochs': 4, 'nminibatches': 4, 'n_timesteps': 10000000.0, 'learning_rate': 'lin_2.5e-4', 'cliprange': 'lin_0.1', 'vf_coef': 0.5, 'ent_coef': 0.01}
8
{'policy': 'MlpPolicy', 'n_envs': 8, 'n_steps': 128, 'noptepochs': 4, 'nminibatches': 4, 'n_timesteps': 10000000.0, 'learning_rate': 'lin_2.5e-4', 'cliprange': 'lin_0.1', 'vf_coef': 0.5, 'ent_coef': 0.01}


In [9]:
from stable_baselines import PPO2, A2C, ACER, ACKTR, DQN, HER, DDPG, TRPO, SAC

ALGOS = {
    'a2c': A2C,
    'acer': ACER,
    'acktr': ACKTR,
    'dqn': DQN,
    'ddpg': DDPG,
    'her': HER,
    'sac': SAC,
    'ppo2': PPO2,
    'trpo': TRPO
}

In [10]:
algo_ = algo = 'ppo2'
normalize = False 
tensorboard_log = './tensorboard_ppo2/'
n_trials = int(1e3)
n_timesteps = int(1e6)
n_jobs = 12
n_envs = 12
sampler = 'random' # one of: 'random','tpe'
pruner = 'halving' # one of: 'halving', 'median', 'none'
seed = int(time.time())

def create_env(n_envs):
            """
            Create the environment
            :param n_envs: (int)
            :return: (gym.Env)
            """
            
            global hyperparams

            if algo_ in ['dqn', 'ddpg']:
                if hyperparams.get('normalize', False):
                    print("WARNING: normalization not supported yet for DDPG/DQN")
                env = gym.make(env_id)
                env.seed(seed)
            else:
                if n_envs == 1:
                    env = DummyVecEnv([make_env(map_size, 0, seed)])
                else:
                    # env = SubprocVecEnv([make_env(env_id, i, args.seed) for i in range(n_envs)])
                    # On most env, SubprocVecEnv does not help and is quite memory hungry
                    env = DummyVecEnv([make_env(map_size, i, seed) for i in range(n_envs)])
                if normalize:
                    if args.verbose > 0:
                        if len(normalize_kwargs) > 0:
                            print("Normalization activated: {}".format(normalize_kwargs))
                        else:
                            print("Normalizing input and reward")
                    env = VecNormalize(env, **normalize_kwargs)
            # Optional Frame-stacking
            if hyperparams.get('frame_stack', False):
                n_stack = hyperparams['frame_stack']
                env = VecFrameStack(env, n_stack)
                print("Stacking {} frames".format(n_stack))
                del hyperparams['frame_stack']
            return env


def create_model(*_args, **kwargs):
                """
                Helper to create a model with different hyperparameters
                """
                return ALGOS[algo](env=create_env(n_envs), tensorboard_log=tensorboard_log,
                                        verbose=0, **kwargs)

In [11]:
if 'n_envs' in hyperparams.keys():
    del hyperparams['n_envs']
if 'n_timesteps' in hyperparams.keys():
    del hyperparams['n_timesteps']

data_frame, best_trial = hyperparam_optimization(algo, create_model, create_env, n_trials=n_trials,
                                                 n_timesteps=n_timesteps, hyperparams=hyperparams,
                                                 n_jobs=n_jobs, sampler_method=sampler, 
                                                 pruner_method=pruner, verbose=1)

Sampler: random - Pruner: halving
Instructions for updating:
Use keras.layers.flatten instead.
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.


[I 2019-06-30 22:36:11,293] Setting status of trial#4 as TrialState.PRUNED. 
[I 2019-06-30 22:36:11,302] Setting status of trial#6 as TrialState.PRUNED. 
[I 2019-06-30 22:36:17,850] Setting status of trial#1 as TrialState.PRUNED. 
[I 2019-06-30 22:36:40,751] Setting status of trial#10 as TrialState.PRUNED. 
[I 2019-06-30 22:36:52,878] Setting status of trial#5 as TrialState.PRUNED. 
[I 2019-06-30 22:38:41,485] Setting status of trial#9 as TrialState.PRUNED. 
[I 2019-06-30 22:40:12,520] Setting status of trial#2 as TrialState.PRUNED. 
[I 2019-06-30 22:40:14,354] Setting status of trial#11 as TrialState.PRUNED. 
[I 2019-06-30 22:40:25,580] Setting status of trial#7 as TrialState.PRUNED. 
[I 2019-06-30 22:46:02,664] Setting status of trial#13 as TrialState.PRUNED. 
[I 2019-06-30 22:46:04,387] Setting status of trial#14 as TrialState.PRUNED. 
[I 2019-06-30 22:46:38,943] Setting status of trial#15 as TrialState.PRUNED. 
[I 2019-06-30 22:46:48,623] Setting status of trial#16 as TrialState.PR

[I 2019-07-01 03:25:58,970] Setting status of trial#77 as TrialState.PRUNED. 
[I 2019-07-01 03:35:59,025] Setting status of trial#79 as TrialState.PRUNED. 
[I 2019-07-01 03:37:08,584] Setting status of trial#80 as TrialState.PRUNED. 
[I 2019-07-01 03:46:39,978] Setting status of trial#82 as TrialState.PRUNED. 
[I 2019-07-01 03:47:53,464] Setting status of trial#83 as TrialState.PRUNED. 
[I 2019-07-01 05:11:14,782] Finished trial#50 resulted in value: 3.9899966716766357. Current best value is 3.9899966716766357 with parameters: {'batch_size': 32, 'n_steps': 2048, 'gamma': 0.999, 'lr': 0.1664672261412471, 'ent_coef': 0.010130230408794483, 'cliprange': 0.4, 'noptepochs': 5, 'lamdba': 0.95}.
[I 2019-07-01 05:12:58,181] Finished trial#45 resulted in value: 3.9899966716766357. Current best value is 3.9899966716766357 with parameters: {'batch_size': 32, 'n_steps': 2048, 'gamma': 0.999, 'lr': 0.1664672261412471, 'ent_coef': 0.010130230408794483, 'cliprange': 0.4, 'noptepochs': 5, 'lamdba': 0.9

[I 2019-07-01 10:00:52,913] Finished trial#102 resulted in value: 3.9899966716766357. Current best value is 3.9899966716766357 with parameters: {'batch_size': 32, 'n_steps': 2048, 'gamma': 0.999, 'lr': 0.1664672261412471, 'ent_coef': 0.010130230408794483, 'cliprange': 0.4, 'noptepochs': 5, 'lamdba': 0.95}.
[I 2019-07-01 10:02:57,214] Finished trial#104 resulted in value: 3.9899966716766357. Current best value is 3.9899966716766357 with parameters: {'batch_size': 32, 'n_steps': 2048, 'gamma': 0.999, 'lr': 0.1664672261412471, 'ent_coef': 0.010130230408794483, 'cliprange': 0.4, 'noptepochs': 5, 'lamdba': 0.95}.
[I 2019-07-01 10:03:15,320] Finished trial#103 resulted in value: 3.9899966716766357. Current best value is 3.9899966716766357 with parameters: {'batch_size': 32, 'n_steps': 2048, 'gamma': 0.999, 'lr': 0.1664672261412471, 'ent_coef': 0.010130230408794483, 'cliprange': 0.4, 'noptepochs': 5, 'lamdba': 0.95}.
[I 2019-07-01 10:13:01,830] Setting status of trial#131 as TrialState.PRUNED

Number of finished trials:  160
Best trial:
Value:  3.9899966716766357
Params: 
    batch_size: 32
    n_steps: 2048
    gamma: 0.999
    lr: 0.1664672261412471
    ent_coef: 0.010130230408794483
    cliprange: 0.4
    noptepochs: 5
    lamdba: 0.95


In [12]:
data_frame.to_csv('./tensorboard_ppo2/dfppo2')

In [ ]:
3.9899966716766357. Current best value is 3.9899966716766357 with parameters: {'batch_size': 32, 'n_steps': 2048, 'gamma': 0.999, 'lr': 0.1664672261412471, 'ent_coef': 0.010130230408794483, 'cliprange': 0.4, 'noptepochs': 5, 'lamdba': 0.95}.

In [15]:
best_trial

FrozenTrial(number=0, state=<TrialState.COMPLETE: 1>, value=3.9899966716766357, datetime_start=datetime.datetime(2019, 6, 30, 22, 26, 30, 262128), datetime_complete=datetime.datetime(2019, 7, 1, 1, 28, 1, 795135), params={'batch_size': 32, 'n_steps': 2048, 'gamma': 0.999, 'lr': 0.1664672261412471, 'ent_coef': 0.010130230408794483, 'cliprange': 0.4, 'noptepochs': 5, 'lamdba': 0.95}, distributions={'batch_size': CategoricalDistribution(choices=(32, 64, 128, 256)), 'n_steps': CategoricalDistribution(choices=(16, 32, 64, 128, 256, 512, 1024, 2048)), 'gamma': CategoricalDistribution(choices=(0.9, 0.95, 0.98, 0.99, 0.995, 0.999, 0.9999)), 'lr': LogUniformDistribution(low=1e-05, high=1), 'ent_coef': LogUniformDistribution(low=1e-08, high=0.1), 'cliprange': CategoricalDistribution(choices=(0.1, 0.2, 0.3, 0.4)), 'noptepochs': CategoricalDistribution(choices=(1, 5, 10, 20, 30, 50)), 'lamdba': CategoricalDistribution(choices=(0.8, 0.9, 0.92, 0.95, 0.98, 0.99, 1.0))}, user_attrs={}, system_attrs={